In [ ]:
pip install transformers

BERT- multilingual tokenizer 는 한국어 단어를 부자연스럽게 쪼갠다

In [ ]:
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
eng_sentence = 'My dog is cute. He likes playing'
print(tokenizer.tokenize(eng_sentence))

In [ ]:
kor_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
kor_sentence = '나는 책상 위에 사과를 먹었다. 알고 보닌 그 사과는 지연의 것이었다. 그래서 지연이에게 사과했다'
print(tokenizer.tokenize(kor_sentence))

활용할 데이터는 영화 커뮤니티의 리뷰 댓글로, 긍정 / 부정에 대한 라벨링이 되어 있다.

In [ ]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

In [ ]:
TEXT = data.Field(batch_first = True,
                  fix_length = 500,
                  tokenize = str.split,
                  pad_first = True,
                  pad_token = '[PAD]',
                  unk_token = '[UNK]')
LABEL = data.LabelField(dtype = torch.float)
train_data, test_data = datasets.IMDB.splits(text_field = TEXT,
                                             label_field = LABEL)

In [ ]:
print('data')
print(' '.join(vars(train_data.examples[1])['text']))
print('label')
print(vars(train_data.examples[1])['label'])

정규표현식을 활용해서 전처리하기

In [ ]:
import re

def PreProcessingText(input_sentence):
    input_sentence = input_sentence.lower() # 소문자화
    input_sentence = re.sub('<[^>]*>', repl= ' ', string = input_sentence) # "<br />" 처리
    input_sentence = re.sub('[!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]', repl= ' ', string = input_sentence) # 특수문자 처리 ("'" 제외)
    input_sentence = re.sub('\s+', repl= ' ', string = input_sentence) # 연속된 띄어쓰기 처리
    if input_sentence:
        return input_sentence

In [ ]:
for example in train_data.examples:
    vars(example)['text'] = PreProcessingText(' '.join(vars(example)['text'])).split()
    
for example in test_data.examples:
    vars(example)['text'] = PreProcessingText(' '.join(vars(example)['text'])).split()

형태소를 추출하여 임베딩하는 모델은 스탠포드에서 학습시킨 'globe.6B.300d'를 활용했다.
내가 하면 등장빈도로 원핫인코딩 정도만 하겠지만.. 
pre_trained vector는 의미를 담은 벡터로 변환한다. https://nlp.stanford.edu/projects/glove/ 

In [ ]:
TEXT.build_vocab(train_data,
                 min_freq = 2,                     # 두번 이상 등장한 단어만 임베딩하겠다. 한번 등장한 단어는 임베딩하지 않겠다.
                 max_size = None,
                 vectors = "glove.6B.300d"
                 )

LABEL.build_vocab(train_data)

In [ ]:
print(f'Vocab Size : {len(TEXT.vocab)}')

print('Vocab Examples : ')
for idx, (k, v) in enumerate(TEXT.vocab.stoi.items()):
    if idx >= 10:
        break    
    print('\t', k, v)

print('---------------------------------')

# Label Info
print(f'Label Size : {len(LABEL.vocab)}')

print('Lable Examples : ')
for idx, (k, v) in enumerate(LABEL.vocab.stoi.items()):
    print('\t', k, v)